# Computing Observables

In [1]:
import sys
import pathlib

# Calculate the path to the src directory
src_path = pathlib.Path('__file__').resolve().parent.parent / 'src'
obs_path = pathlib.Path('__file__').resolve().parent.parent / "notebooks" / "observables"


# Add src directory to sys.path
for path in (src_path, obs_path):
    if src_path not in sys.path:
        sys.path.append(str(path))


# Verify that the path has been added
print(f"sys.path: {sys.path}")

sys.path: ['/Users/andrewprojansky/Documents/GitHub/LMAlgolabs/notebooks', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python39.zip', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/lib-dynload', '', '/Users/andrewprojansky/Documents/GitHub/LMAlgolabs/venv/lib/python3.9/site-packages', '/Users/andrewprojansky/Documents/GitHub/LMAlgolabs/src', '/Users/andrewprojansky/Documents/GitHub/LMAlgolabs/notebooks/observables']


In this exercise, you will be sampling from an already trained model and to compute observables. A class `Autoload` has been created for you; it comes from the `observables` module. When instantiated, this object loads saved parameters from the `saved_params`. 

- It provides a `sample` method that can be used to sample from the network. By default, it would return 1000 samples but you can specify any sample size $N_s$ that you want as a function argument, `sample(N_s)`.
- It also provides a method `logpsi` for computing the log of your squared wave function amplitude.

Note that the sample that is returned is a Jax array implementation. If you are not familiar with Jax, you can convert the output to the numpy format by calling `numpy.asarray()` and passing the Jax array as input.

In [4]:
from observables import Autoload

In [5]:
# Sampling from the trained network using Autoload

nn_state = Autoload()
samples = nn_state.sample()
print(f"{samples.shape}, \n\n {samples}")

(1000, 16), 

 [[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 1 0]
 [1 1 0 ... 1 0 1]
 [0 1 0 ... 0 0 0]]


Using the generated samples, you can compute whatever observable that you are interested in. An example would be the average magnetization in the $z$-direction, $\left\langle\hat{\sigma}^z\right\rangle=\frac{1}{N_s N}\sum_{q=1}^{N_s}\sum_{i=1}^N\left\langle\hat{\sigma}_i^z\right\rangle_q$, where we average over the $N=4\times 4$ atoms in the system and the $N_s$ created samples.
The evaluate of $\left\langle\hat{\sigma}^z\right\rangle$ is shown below:

In [6]:
from jax import numpy as jnp

In [7]:
# Start by coverting the qubit representation from the states (0, 1) to the states (-1, 1)
# This needs to be done to match the definition of the magnetization as the eigenvalues of the Pauli z matrix
my_samples = nn_state.sample(500)
my_conv_samples =  2 * jnp.array(my_samples) - 1
print(my_conv_samples)

[[ 1  1 -1 ...  1 -1  1]
 [-1 -1 -1 ... -1 -1  1]
 [ 1 -1 -1 ...  1 -1 -1]
 ...
 [-1 -1 -1 ...  1 -1 -1]
 [ 1 -1  1 ... -1 -1  1]
 [-1 -1 -1 ... -1 -1  1]]


In [8]:
# Now compute the average magnetization
jnp.mean(jnp.mean(my_conv_samples, axis = 1)).item()

-0.5097500085830688

We can now also create a class that inherits from the class `Observable`, which is provided in the `observables` module.
For this we need to do the following steps:

- Compulsorily define a `compute` instance method in which we define the implementation
- Optionally define a class name and a symbol

Let us use the just completed magnetization as an example and create a class called `ZMagnetization`:

In [9]:
from observables import Observable

class ZMagnetization(Observable):
    def __init__(self):
        self.name = "SigmaZ"
        self.symbol = "Z"

    def compute(self, model, sample_size):
        samps = model.sample(sample_size)
        conv_samples = 2 * jnp.array(samps) - 1

        ave_mag = jnp.mean(jnp.mean(conv_samples, axis = 1))

        return ave_mag.item()
        

Now we can instantiate the class and ask it to compute the magnetization with the trained network using any sample size $N_s$:

In [12]:
magnet_z = ZMagnetization()
magnet_z.compute(model = nn_state, sample_size = 2000)

-0.5022500157356262

## YOUR TASKS - Compute Off-Diagonal Observables

1. Create a class that inherits `Observable` and can compute the expectation value of the in-plane magnetization - $\left \langle \sigma^x \right \rangle = \frac{1}{N_s N}\sum_{q=1}^{N_s}\sum_{i=1}^N\left\langle\hat{\sigma}_i^x\right\rangle_q$
2. Create a class that inherits `Observable` and can compute the second order Rényi entropy $S_2$ as a function of the size of a sub-region $A$.
   The second order Rényi entropy for the state with density matrix $\rho_A$ on the sub-region $A$ is defined as
   $S_2\left(\hat{\rho}_A\right) = - \mathrm{ln}\left[\mathrm{tr}\left(\hat{\rho}_A^2\right)\right]$ and can be evaluated by introducing the swap operator $\mathrm{Swap}_A$. Here are the individual steps to evaluate the second order Rényi entropy:
   - Construct two copies of the system
   - Divide the state into two sub-regions $A$ and $B$: $\left|\Psi\right\rangle = \sum_{\alpha, \beta}C_{\alpha,\beta}\left|\alpha\right\rangle\otimes\left|\beta\right\rangle$ with wave function amplitudes $C_{\alpha, \beta}$
   - Define the swap operator acting on the system including the two copies:
     $\mathrm{Swap}_A\left[\sum_{\alpha_1,\beta_1}C_{\alpha_1,\beta_1}\left|\alpha_1\right\rangle\otimes\left|\beta_1\right\rangle\right]\otimes\left[\sum_{\alpha_2,\beta_2}D_{\alpha_2,\beta_2}\left|\alpha_2\right\rangle\otimes\left|\beta_2\right\rangle\right] = 
     \sum_{\alpha_1,\beta_1}C_{\alpha_1,\beta_2}\sum_{\alpha_2,\beta_2}D_{\alpha_2,\beta_2}\left[\left|\alpha_2\right\rangle\otimes\left|\beta_1\right\rangle\right]\otimes\left[\left|\alpha_1\right\rangle\otimes\left|\beta_2\right\rangle\right]$

   - The second order Rényi entropy is then defined as $S_2\left(\hat{\rho}_A\right)=-\mathrm{ln}\left[\left\langle\mathrm{Swap}_A\right\rangle\right]$, where the expectation value $\left\langle\mathrm{Swap}_A\right\rangle=\left\langle\Psi\left|\mathrm{Swap}_A\right|\Psi\right\rangle$ can be evaluated using the $N_s$ samples generated from the language model.

   You can thus implement the Swap function and apply it to the samples generated from the pre-trained RNN to get an estimate of the second order Rényi entropy. Start with choosing the sub-region $A$ as the left array of $4\times 2$ atoms. Once implemented, you can vary this sub-region $A$. The sub-region $B$ always corresponds to all atoms that are not in $A$.

In [14]:
from observables import Observable

'''
I'm gonna do both of these in psuedocode. I am a bit tired, and also it feels like I do not have the allotted time to fully code these things 
fully. However, I do think coding them, or least writing what I believe to be psuedo-code, is important. 
'''

'''
X Observable 


class XMagnetization(Observable):
    def __init__(self):
        self.name = "SigmaX"
        self.symbol = "X"

    def compute(self, model, sample_size):
        val = 0
        h_dict = {}
        samps = model.sample(sample_size)
        for val in samps: 
            if val not in h_dict.keys():
                h_dict[val] = 1
            else:
                h_dict[val] += 1

        for ind in range(n):
            for keys in h_dict:
                newkey = copy key
                newkey[ind] = (keys[ind]+1)%2 
                if newkeys in h_dict.keys():
                    val += h_dict[keys]/h_dict{newkeys}

        return val/len(samps)

'''

'''
Second Renyii Entropy

class SecondRenyi(Observable):
    def __init__(self):
        self.name = "Swap"
        self.symbol = "S"

    def compute(self, model, sample_size):
        val = 0
        h_dict = {}
        samps = model.sample(sample_size)
        n = len(samps[0])
        sampsA = []
        sampsB = []
        for samp in samps:
            sampsA.append(samps[0:N//2])
            sampsB.append(samps[N//2:])

        do swap test on double copy 

        take expectation values
        
        

'''

'\nX Observable \n\n\nclass XMagnetization(Observable):\n    def __init__(self):\n        self.name = "SigmaX"\n        self.symbol = "X"\n\n    def compute(self, model, sample_size):\n        val = 0\n        h_dict = {}\n        samps = model.sample(sample_size)\n        for val in samps: \n            if val not in h_dict.keys():\n                h_dict[val] = 1\n            else:\n                h_dict[val] += 1\n\n        for ind in range(n):\n            for keys in h_dict:\n\n        ave_mag = jnp.mean(jnp.mean(conv_samples, axis = 1))\n\n        return ave_mag.item()\n\n'